# 循环神经网络

在上一个模块中，我们一直在使用文本的丰富语义表示，以及在嵌入之上的简单线性分类器。这种架构的作用是捕获句子中单词的聚合含义，但它没有考虑单词的**顺序**，因为嵌入之上的聚合操作从原始文本中删除了这些信息。由于这些模型无法对词序进行建模，因此它们无法解决更复杂或模棱两可的任务，例如文本生成或问答。

为了捕捉文本序列的含义，我们需要使用另一种神经网络架构，称为**循环神经网络**，或 RNN。在 RNN 中，我们一次一个符号通过网络，网络产生一些**状态**，然后我们用下一个符号再次传递给网络。

![显示循环神经网络生成示例的图像。](./images/sample-rnn-model-generation.png)

给定令牌 $X_0,\dots,X_n$ 的输入序列，RNN 创建一个神经网络块序列，并使用反向传播端到端地训练这个序列。每个网络块以一对 $(X_i,S_i)$ 作为输入，并产生 $S_{i+1}$ 作为结果。最终状态 $S_n$ 或输出 $X_n$ 进入线性分类器以产生结果。所有网络块共享相同的权重，并使用一次反向传播进行端到端的训练。

因为状态向量 $S_0,\dots,S_n$ 是通过网络传递的，所以它能够学习单词之间的顺序依赖关系。例如，当单词 *not* 出现在序列中的某处时，它可以学习否定状态向量中的某些元素，从而产生否定。

让我们看看循环神经网络如何帮助我们对新闻数据集进行分类。

In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## 简单的RNN分类器

在简单 RNN 的情况下，每个循环单元都是一个简单的线性网络，它采用连接的输入向量和状态向量，并产生一个新的状态向量。 PyTorch 用“RNNCell”类和此类单元的网络表示该单元——作为“RNN”层。

为了定义一个 RNN 分类器，我们将首先应用一个嵌入层来降低输入词汇的维数，然后在它上面有一个 RNN 层：

In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **注意：** 为简单起见，我们在这里使用未经训练的嵌入层，但为了获得更好的结果，我们可以使用带有 Word2Vec 或 GloVe 嵌入的预训练嵌入层，如前一单元所述。为了更好地理解，您可能需要修改此代码以使用预训练的嵌入。

在我们的例子中，我们将使用填充数据加载器，因此每个批次都会有许多相同长度的填充序列。 RNN 层将采用嵌入张量的序列，并产生两个输出：
* $x$ 是每一步的 RNN 单元输出序列
* $h$ 是序列最后一个元素的最终隐藏状态

然后我们应用一个全连接的线性分类器来获得类的数量。

> **注意：** RNN 很难训练，因为一旦 RNN 单元沿着序列长度展开，反向传播所涉及的层数就会非常大。因此我们需要选择较小的学习率，并在较大的数据集上训练网络以产生良好的结果。这可能需要很长时间，因此首选使用 GPU。

In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.315625
6400: acc=0.39109375
9600: acc=0.4557291666666667
12800: acc=0.5075
16000: acc=0.5468125
19200: acc=0.5794270833333334
22400: acc=0.6034375
25600: acc=0.6250390625
28800: acc=0.6442361111111111
32000: acc=0.66009375
35200: acc=0.6740625
38400: acc=0.6864583333333333
41600: acc=0.6972836538461539
44800: acc=0.7069866071428571
48000: acc=0.71575
51200: acc=0.723359375
54400: acc=0.7304779411764706
57600: acc=0.7376388888888888
60800: acc=0.743125
64000: acc=0.74840625
67200: acc=0.7534821428571429
70400: acc=0.7587073863636363
73600: acc=0.763546195652174
76800: acc=0.7678645833333333
80000: acc=0.77175
83200: acc=0.7753605769230769
86400: acc=0.7786574074074074
89600: acc=0.782109375
92800: acc=0.7851831896551724
96000: acc=0.7882395833333333
99200: acc=0.7908467741935484
102400: acc=0.79353515625
105600: acc=0.796032196969697
108800: acc=0.7986948529411765
112000: acc=0.8009375
115200: acc=0.8029861111111111
118400: acc=0.8049155405405405


(0.03329115804036458, 0.8059666666666667)

## 长短期记忆（LSTM）

经典 RNN 的主要问题之一是所谓的 **消失梯度** 问题。由于 RNN 是在一次反向传播中端到端训练的，因此很难将错误传播到网络的第一层，因此网络无法学习远距离标记之间的关系。避免这个问题的方法之一是通过使用所谓的**门**来引入**显式状态管理**。有两种最著名的此类架构：**长短期记忆** (LSTM) 和**门控中继单元** (GRU)。

![显示长短期记忆单元示例的图像](./images/long-short-term-memory-cell.svg)

LSTM 网络的组织方式类似于 RNN，但有两种状态在层与层之间传递：实际状态 $c$ 和隐藏向量 $h$。在每个单元，隐藏向量 $h_i$ 与输入 $x_i$ 连接，它们通过 **gates** 控制状态 $c$ 发生的事情。每个门都是一个带有 sigmoid 激活的神经网络（输出在 $[0,1]$ 范围内），当乘以状态向量时，可以将其视为按位掩码。有以下几个门（上图从左到右）：
* **forget gate** 获取隐藏向量并确定我们需要忘记向量 $c$ 的哪些组件，以及要通过哪些组件。
* **输入门**从输入和隐藏向量中获取一些信息，并将其插入到状态中。
* **输出门**通过一些带有$\tanh$激活的线性层来转换状态，然后使用隐藏向量$h_i$选择它的一些组件来产生新的状态$c_{i+1}$。

状态 $c$ 的组件可以被认为是一些可以打开和关闭的标志。例如，当我们在序列中遇到一个名字 *Alice* 时，我们可能想假设它指的是女性角色，并在句子中有女性名词的状态下升旗。当我们进一步遇到短语*和Tom*时，我们会举起我们有复数名词的标志。因此，通过操纵状态，我们应该可以跟踪句子部分的语法特性。

> **注意**：Christopher Olah 撰写的这篇很棒的文章 [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) 是了解 LSTM 内部结构的重要资源。

虽然 LSTM 单元的内部结构可能看起来很复杂，但 PyTorch 将这个实现隐藏在 LSTMCell 类中，并提供了 LSTM 对象来表示整个 LSTM 层。因此，LSTM 分类器的实现将与我们上面看到的简单 RNN 非常相似：

In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

现在让我们训练我们的网络。 请注意，训练 LSTM 也很慢，而且在训练开始时，您的准确率可能没有太大提高。 此外，您可能需要使用 `lr` 学习率参数来找到导致合理训练速度的学习率，但不会导致

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.2578125
6400: acc=0.258125
9600: acc=0.2575
12800: acc=0.2571875
16000: acc=0.257625
19200: acc=0.25916666666666666
22400: acc=0.26071428571428573
25600: acc=0.2833203125
28800: acc=0.3047916666666667
32000: acc=0.33015625
35200: acc=0.358125
38400: acc=0.386015625
41600: acc=0.41259615384615383
44800: acc=0.43828125
48000: acc=0.4630416666666667
51200: acc=0.485078125
54400: acc=0.5053492647058824
57600: acc=0.5237326388888889
60800: acc=0.5408059210526316
64000: acc=0.55575
67200: acc=0.5701488095238095
70400: acc=0.5834517045454546
73600: acc=0.595203804347826
76800: acc=0.606640625
80000: acc=0.617375
83200: acc=0.6270913461538462
86400: acc=0.6360185185185185
89600: acc=0.6450223214285714
92800: acc=0.6529741379310345
96000: acc=0.6605729166666666
99200: acc=0.6679334677419355
102400: acc=0.67458984375
105600: acc=0.6809280303030303
108800: acc=0.6872518382352941
112000: acc=0.6930178571428571
115200: acc=0.6981597222222222
118400: acc=0.7032939189189189


(0.041912955729166666, 0.7057333333333333)

## 打包序列

在我们的示例中，我们必须用零向量填充小批量中的所有序列。虽然这会导致一些内存浪费，但对于 RNN，更重要的是为填充的输入项创建额外的 RNN 单元，这些单元参与训练，但不携带任何重要的输入信息。仅将 RNN 训练到实际序列大小会好得多。

为此，PyTorch 中引入了一种特殊的填充序列存储格式。假设我们有输入的填充小批量，它看起来像这样：
``
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
``
这里 0 表示填充值，输入序列的实际长度向量为`[5,3,1]`。

为了用填充序列有效地训练 RNN，我们想开始用大 minibatch (`[1,6,9]`) 训练第一组 RNN 单元，然后结束第三个序列的处理，并继续用短小批量 ( `[2,7]`、`[3,8]`）等。因此，打包序列被表示为一个向量——在我们的例子中是`[1,6,9,2,7,3,8,4,5]`，和长度向量（`[5,3,1]`），从中我们可以轻松地重建原始的填充小批量。

要生成打包序列，我们可以使用 `torch.nn.utils.rnn.pack_padd_sequence` 函数。所有循环层，包括 RNN、LSTM 和 GRU，都支持打包序列作为输入，并生成打包输出，可以使用 `torch.nn.utils.rnn.pad_packed_sequence` 进行解码。

为了能够生成打包序列，我们需要将长度向量传递给网络，因此我们需要一个不同的函数来准备小批量：

In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

实际网络与上面的“LSTMClassifier”非常相似，但“forward”传递将同时接收填充的小批量和序列长度向量。 计算嵌入后，我们计算打包序列，将其传递给 LSTM 层，然后将结果解包回来。

> **注意**：我们实际上不使用解包结果`x`，因为我们在下面的计算中使用了隐藏层的输出。 因此，我们可以从这段代码中完全删除解包。 我们把它放在这里的原因是为了让您能够轻松修改此代码，以防您需要在进一步计算中使用网络输出。

In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

现在让我们进行训练

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.2696875
6400: acc=0.32375
9600: acc=0.3840625
12800: acc=0.439765625
16000: acc=0.489375
19200: acc=0.53109375
22400: acc=0.5648660714285715
25600: acc=0.593515625
28800: acc=0.6190277777777777
32000: acc=0.63934375
35200: acc=0.6569602272727273
38400: acc=0.67296875
41600: acc=0.6859375
44800: acc=0.6974553571428571
48000: acc=0.708625
51200: acc=0.718203125
54400: acc=0.7272977941176471
57600: acc=0.7352951388888889
60800: acc=0.7425493421052631
64000: acc=0.748671875
67200: acc=0.7548660714285714
70400: acc=0.76078125
73600: acc=0.7656657608695652
76800: acc=0.7706380208333333
80000: acc=0.7746125
83200: acc=0.7789903846153846
86400: acc=0.782974537037037
89600: acc=0.7871205357142858
92800: acc=0.7904310344827586
96000: acc=0.7939479166666666
99200: acc=0.7971572580645161
102400: acc=0.8003125
105600: acc=0.803030303030303
108800: acc=0.8057261029411765
112000: acc=0.8083035714285715
115200: acc=0.81078125
118400: acc=0.8132010135135135


(0.0298255859375, 0.81445)

> **注意：** 您可能已经注意到我们传递给训练函数的参数 `use_pack_sequence`。 目前，`pack_padded_sequence` 函数要求长度序列张量位于 CPU 设备上，因此训练函数在训练时需要避免将长度序列数据移动到 GPU。 您可以在 [`torchnlp.py`](torchnlp.py) 文件中查看 `train_emb` 函数的实现。

## 双向和多层 RNN

在我们的示例中，所有循环网络都在一个方向上运行，从序列的开始到结束。它看起来很自然，因为它类似于我们阅读和聆听演讲的方式。然而，由于在许多实际情况下我们可以随机访问输入序列，因此在两个方向上运行循环计算可能是有意义的。这种网络被称为 **bidirectional** RNN，它们可以通过将 `bidirectional=True` 参数传递给 RNN/LSTM/GRU 构造函数来创建。

在处理双向网络时，我们需要两个隐藏状态向量，每个方向一个。 PyTorch 将这些向量编码为一个两倍大的向量，这非常方便，因为您通常会将生成的隐藏状态传递给完全连接的线性层，并且您只需要在创建层时考虑这种大小的增加.

循环网络，单向或双向，捕获序列中的某些模式，并且可以将它们存储到状态向量中或传递到输出中。与卷积网络一样，我们可以在第一个循环层之上构建另一个循环层来捕获更高级别的模式，从第一层提取的低级模式构建。这引出了**多层 RNN** 的概念，它由两个或多个循环网络组成，其中前一层的输出作为输入传递到下一层。

![显示多层长短期记忆 RNN 的图像](./images/multi-layer-lstm.jpeg)

*Fernando López 来自 [这篇精彩文章](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) 的图片*

PyTorch 使构建此类网络成为一项简单的任务，因为您只需要将 `num_layers` 参数传递给 RNN/LSTM/GRU 构造函数即可自动构建多层循环。这也意味着隐藏/状态向量的大小会按比例增加，并且在处理循环层的输出时需要考虑到这一点。

## 用于其他任务的 RNN

在本单元中，我们已经看到 RNN 可用于序列分类，但实际上，它们可以处理更多任务，例如文本生成、机器翻译等。 我们将在下一个单元中考虑这些任务。